In [1]:
import random
import sys
import time

In [2]:
from pyspark import SparkConf
from pyspark import SparkContext
conf = SparkConf().setMaster("local").setAppName("OneMaxPython")
sc = SparkContext(conf = conf)

In [3]:
class Individual:
    def __init__(self, chromosome, fitness=None):
        # 初始化函数
        self.chromosome = chromosome
        self.fitness = fitness

    def mutate(self, mutation): # 基因突变函数，参数 mutation 是基因变异概率
        for i in self.chromosome: # 遍历每个遗传因子，准备变异
            if mutation > (random.randint(0, 100) / 100): # 大于变异概率就进行变异
                self.chromosome[i] = random.randint(0, 1)

    def fit(self):
        self.fitness = sum(self.chromosome) / len(self.chromosome) # 适应值为1的百分比，最好为100%

In [4]:
def create_individual(chr_size): # 创建个体, 返回一个个体类
    return Individual([random.randint(0, 1) for i in range(chr_size)])

In [5]:
def MakePop(pop_size, ind_size):
    lisPop = []
    for i in range(pop_size):
        lisPop.append([[random.randint(0,1) for i in range(ind_size)], 0])
    return lisPop

In [6]:
random.seed(64) # 随机种子设置

CHROMOSOME_SIZE = 10 # 染色体尺寸
GENE_MUTATION = 0.05 # 基因变异率
INDIVIDUAL_MUTATION = 0.2 # 个体变异率
CROSSOVER = 0.5
POPULATION_SIZE = 40 # 种群数量
ELITE_PERCENTAGE = 0.5
#ELITE_SIZE = int(POPULATION_SIZE * ELITE_PERCENTAGE)
ELITE_SIZE = 2 
GENERATION_MAX = 100 # 最大迭代次数

In [8]:
start = time.clock() # 开始计时
population = MakePop(POPULATION_SIZE, CHROMOSOME_SIZE)
popRDD = sc.parallelize(population)
    # 计算每个个体的适应度
   # fitPopRDD = popRDD.map(FitnessForEachInd)
    #fitPopRDD.count() 

In [10]:
def Fitness(ind):
    return sum(ind[0])/len(ind[0])
def EvaForEachInd(ele):
    ele[1] = sum(ele[0])/len(ele[0])
    return ele
    

In [11]:
# 以下开始是循环内容
# 计算每个个体的适应度
fitRDD = popRDD.map(EvaForEachInd)

In [12]:
# 把所有的Fitness都取出来
fitValues = [ele[1] for ele in fitRDD.collect()]

In [13]:
print("{0} Generation ---".format(1))
print("\tMIN: {0}".format(min(fitValues)))
print("\tMAX: {0}".format(max(fitValues)))
print("\tAVG: {0}".format(round(sum(fitValues) / len(fitValues), 3)), "\n")

1 Generation ---
	MIN: 0.2
	MAX: 0.8
	AVG: 0.495 



In [14]:
#选择精英者和剩下的群体
def Select(fitRDD, POPULATION_SIZE, ELITE_SIZE):
# 对评价过的种群按适应度从大到小进行排序
    sortedPopRDD = fitRDD.sortBy((lambda x: x[1]), False)
# 取出精英并创建精英RDD
    lisElite = sortedPopRDD.take(ELITE_SIZE)
    eliteRDD = sc.parallelize(lisElite)
# 取出剩下的种群
    RemainPop = fitRDD.sortBy(lambda x: x[1]).take(POPULATION_SIZE - ELITE_SIZE)
    random.shuffle(RemainPop)
    RemainPopRDD = sc.parallelize(RemainPop)
    return eliteRDD, RemainPopRDD

In [15]:
# 选择精英者和剩下的群体
eliteRDD, RemainPopRDD = Select(fitRDD, POPULATION_SIZE, ELITE_SIZE)

In [16]:
# 交叉部分
# 交叉函数
RemainPopList = RemainPopRDD.collect()


In [28]:
PairRDD = sc.parallelize(RemainPopList, len(RemainPopList)/2).glom()

In [41]:
def crossover(a, b): # 两个个体交叉操作
    # Two-point crossover
    a_chr = a # 取出a的染色体
    b_chr = b # 取出b的染色体
    
    size = len(a_chr) # 取出染色体的长度

    f = random.randint(0, size) # 选取两个基因点，准备交叉
    s = random.randint(f, size)

    _a = a_chr[:f] + b_chr[f:s] + a_chr[s:]
    _b = b_chr[:f] + a_chr[f:s] + b_chr[s:]
    
    return _a, _b

def CROSSOVER(ele):
    a_Chromo, b_Chromo = crossover(ele[0][0],ele[1][0])
    ele[0][0] = a_Chromo
    ele[0][1] = 0
    ele[1][0] = b_Chromo
    ele[1][1] = 0
    return ele[0],ele[1]

In [45]:
CrossedRDD = PairRDD.flatMap(CROSSOVER)

In [46]:
CrossedList = CrossedRDD.collect()
random.shuffle(CrossedList)
CrossedRDD = sc.parallelize(CrossedList)

In [67]:
len(RemainPopList)/2

19.0

In [54]:
# 变异部分
def MutationForInd(gene):
    global GENE_MUTATION
    for i in gene:
        if GENE_MUTATION > (random.randint(0, 100) / 100):
            i = random.randint(0,1)
    return gene
            
def Mutation(ele):
    global INDIVIDUAL_MUTATION
    if INDIVIDUAL_MUTATION > (random.randint(0, 100) / 100):
        ele[0] = MutationForInd(ele[0])
    return ele
    


In [55]:
MutatedRDD = CrossedRDD.map(Mutation)

In [62]:
MutatedList = MutatedRDD.collect()
eliteList = eliteRDD.collect()
population = MutatedList + eliteList

In [64]:
popRDD = sc.parallelize(population)

[[[1, 0, 1, 0, 0, 1, 0, 0, 1, 1], 0],
 [[1, 0, 0, 0, 0, 1, 1, 0, 0, 0], 0],
 [[1, 1, 1, 1, 1, 0, 0, 0, 0, 1], 0],
 [[0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 0],
 [[1, 0, 1, 0, 0, 1, 0, 0, 0, 0], 0],
 [[1, 1, 0, 1, 0, 1, 1, 0, 0, 1], 0],
 [[1, 1, 1, 1, 0, 1, 1, 1, 0, 0], 0],
 [[1, 0, 0, 1, 1, 1, 1, 0, 1, 1], 0],
 [[1, 1, 0, 0, 1, 0, 0, 1, 1, 1], 0],
 [[1, 1, 1, 1, 0, 0, 0, 0, 1, 0], 0],
 [[1, 0, 0, 0, 1, 0, 0, 0, 0, 0], 0],
 [[0, 0, 1, 1, 0, 0, 1, 1, 1, 1], 0],
 [[0, 0, 1, 1, 0, 1, 1, 1, 0, 1], 0],
 [[0, 0, 1, 0, 0, 1, 0, 1, 0, 0], 0],
 [[0, 0, 1, 0, 0, 1, 1, 0, 0, 1], 0],
 [[0, 0, 0, 0, 0, 0, 1, 1, 0, 0], 0],
 [[0, 1, 1, 1, 0, 0, 0, 1, 1, 0], 0],
 [[1, 0, 0, 1, 1, 1, 0, 0, 0, 0], 0],
 [[0, 1, 1, 0, 0, 0, 1, 0, 1, 0], 0],
 [[0, 0, 1, 1, 1, 1, 0, 1, 1, 1], 0],
 [[1, 0, 1, 1, 0, 0, 1, 0, 1, 0], 0],
 [[1, 0, 1, 1, 1, 1, 1, 0, 0, 1], 0],
 [[1, 1, 1, 0, 1, 0, 1, 0, 1, 0], 0],
 [[0, 1, 1, 0, 1, 0, 0, 1, 1, 0], 0],
 [[0, 0, 0, 1, 1, 1, 1, 0, 1, 1], 0],
 [[0, 1, 1, 1, 1, 0, 1, 1, 0, 1], 0],
 [[0, 0, 1, 